In [38]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans  
import json
import pprint
from pathlib import Path

## Fetching Cryptocurrency Data

In [39]:
# Alternatively, use the provided csv file:
# Create a DataFrame 
path = Path('Resources/crypto_data.csv')
crypto_df = pd.read_csv(path, index_col = 'Unnamed: 0')

In [40]:
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,3.14159E+11
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


## Data Preprocessing

In [41]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [42]:
# Keep only cryptocurrencies with a working algorithm 
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [43]:
# Remove the "IsTrading" column
crypto_df.drop(columns = ['IsTrading'], inplace = True)

In [44]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace = True)

In [45]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [46]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName_df = crypto_df[['CoinName']]

In [47]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns = ['CoinName'], inplace = True)

In [48]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns = ['Algorithm','ProofType'])

In [49]:
# Standardize data
scaler = StandardScaler()

crypto_scaled = scaler.fit_transform(X)

## Reducing Dimensions Using PCA
 

In [50]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [51]:
# Create a DataFrame with the principal components data
pcs = pca.fit_transform(crypto_scaled)

pcs_df = pd.DataFrame(data = pcs, columns = ['PC 1', 'PC 2', 'PC 3'], index = crypto_df.index)
pcs_df

,PC 1,PC 2,PC 3
42,-0.322301,0.968221,-0.586033
404,-0.305655,0.968523,-0.586616
1337,2.291992,1.678473,-0.752626
BTC,-0.157049,-1.290436,0.199899
ETH,-0.162365,-2.000445,0.403730
...,...,...,...
ZEPH,2.484276,0.778252,-0.198855
GAP,-0.320350,0.968130,-0.586070
BDX,0.305870,-2.226958,0.424959
ZEN,-0.160675,-1.981316,0.441323


## Clustering Crytocurrencies Using K-Means

In [52]:
# Find the Best Value for k Using the Elbow Curve
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\javie\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

## Running K-Means with k= <your best value for k here> 

In [53]:
# Initialize the K-Means model
model = KMeans(n_clusters=3, random_state=5)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = crypto_df.join(pcs_df)
clustered_df['CoinName'] = CoinName_df['CoinName']
clustered_df['Class'] = predictions
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.322301,0.968221,-0.586033,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.305655,0.968523,-0.586616,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11,2.291992,1.678473,-0.752626,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.157049,-1.290436,0.199899,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.162365,-2.000445,0.403730,Ethereum,0


## Visualizing Results

In [54]:
#3D-Scatter with Clusters
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

## Table of Tradable Cryptocurrencies

In [55]:
# Table with tradable cryptos
clustered_df[['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply', 'CoinName', 'Class']].hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,Class]

## Scatter Plot with Tradable Cryptocurrencies

In [56]:
# Scale data to create the scatter plot
mm_scaler = MinMaxScaler()
data = mm_scaler.fit_transform(
    clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]
)
data_df = pd.DataFrame(
    data, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index
)
data_df["CoinName"] = clustered_df["CoinName"]
data_df["Class"] = clustered_df["Class"]
data_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,0
404,5.320000e-04,0.007002,404Coin,0
1337,3.141590e-01,0.035342,EliteCoin,0
BTC,2.100000e-05,0.005960,Bitcoin,0
ETH,0.000000e+00,0.006050,Ethereum,0


In [57]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
data_df.hvplot.scatter(
    x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"], by="Class"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)